In [1]:
using CSV, DataFrames, StatsBase, Clustering

In [2]:
df = CSV.read("localization.csv", header=false)

┌ Warning: `CSV.read(input; kw...)` is deprecated in favor of `DataFrame!(CSV.File(input; kw...))`
└ @ CSV /Users/xronos/.julia/packages/CSV/OM6FO/src/CSV.jl:40


,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,-64,-56,-61,-66,-71,-82,-81,1
2,-68,-57,-61,-65,-71,-85,-85,1
3,-63,-60,-60,-67,-76,-85,-84,1
4,-61,-60,-68,-62,-77,-90,-80,1
5,-63,-65,-60,-63,-77,-81,-87,1
6,-64,-55,-63,-66,-76,-88,-83,1
7,-65,-61,-65,-67,-69,-87,-84,1
8,-61,-63,-58,-66,-74,-87,-82,1
9,-65,-60,-59,-63,-76,-86,-82,1


In [3]:
old_names = names(df)
new_names = map(Symbol, ["WiFi1", "WiFi2", "WiFi3", "WiFi4", "WiFi5", "WiFi6", "WiFi7", "Room"])
for i = 1:8
    rename!(df, old_names[i] => new_names[i])
end

In [4]:
df[:RegressionTarget] = Matrix(df[[1, 4, 6, 7]]) * [5, 10, 15, 20] + 0.01*randn(2000)

┌ Warning: `getindex(df::DataFrame, col_inds::Union{AbstractVector, Regex, Not})` is deprecated, use `df[:, col_inds]` instead.
│   caller = top-level scope at In[4]:1
└ @ Core In[4]:1
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = top-level scope at In[4]:1
└ @ Core In[4]:1


2000-element Array{Float64,1}:
 -3829.991539220298
 -3965.0072762078016
 -3939.9966445066198
 -3875.0233164425745
 -3899.990675008289
 -3960.0119978467715
 -3980.000951770705
 -3910.015336896875
 -3885.004020627609
 -4099.995839470785
 -4069.988587771487
 -3904.996285535618
 -3904.9957157080976
     ⋮
 -3815.0134960880796
 -3655.00267610889
 -3780.0140722546043
 -3719.989828215735
 -3970.02070221255
 -4010.0046548800697
 -4000.0063297762918
 -4125.011973427495
 -4020.0057416450572
 -4025.008162103762
 -3969.9733601435323
 -3954.975637470834

In [5]:
first(df, 10)

,WiFi1,WiFi2,WiFi3,WiFi4,WiFi5,WiFi6,WiFi7,Room,RegressionTarget
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64
1,-64,-56,-61,-66,-71,-82,-81,1,-3829.99
2,-68,-57,-61,-65,-71,-85,-85,1,-3965.01
3,-63,-60,-60,-67,-76,-85,-84,1,-3940.0
4,-61,-60,-68,-62,-77,-90,-80,1,-3875.02
5,-63,-65,-60,-63,-77,-81,-87,1,-3899.99
6,-64,-55,-63,-66,-76,-88,-83,1,-3960.01
7,-65,-61,-65,-67,-69,-87,-84,1,-3980.0
8,-61,-63,-58,-66,-74,-87,-82,1,-3910.02
9,-65,-60,-59,-63,-76,-86,-82,1,-3885.0


In [6]:
# Turn all the values into float64
XX = map(Float64, Matrix(df[1:7]))

┌ Warning: `getindex(df::DataFrame, col_inds::Union{AbstractVector, Regex, Not})` is deprecated, use `df[:, col_inds]` instead.
│   caller = top-level scope at In[6]:1
└ @ Core In[6]:1


2000×7 Array{Float64,2}:
 -64.0  -56.0  -61.0  -66.0  -71.0  -82.0  -81.0
 -68.0  -57.0  -61.0  -65.0  -71.0  -85.0  -85.0
 -63.0  -60.0  -60.0  -67.0  -76.0  -85.0  -84.0
 -61.0  -60.0  -68.0  -62.0  -77.0  -90.0  -80.0
 -63.0  -65.0  -60.0  -63.0  -77.0  -81.0  -87.0
 -64.0  -55.0  -63.0  -66.0  -76.0  -88.0  -83.0
 -65.0  -61.0  -65.0  -67.0  -69.0  -87.0  -84.0
 -61.0  -63.0  -58.0  -66.0  -74.0  -87.0  -82.0
 -65.0  -60.0  -59.0  -63.0  -76.0  -86.0  -82.0
 -62.0  -60.0  -66.0  -68.0  -80.0  -86.0  -91.0
 -67.0  -61.0  -62.0  -67.0  -77.0  -83.0  -91.0
 -65.0  -59.0  -61.0  -67.0  -72.0  -86.0  -81.0
 -63.0  -57.0  -61.0  -65.0  -73.0  -84.0  -84.0
   ⋮                                  ⋮    
 -57.0  -55.0  -50.0  -57.0  -49.0  -84.0  -85.0
 -56.0  -55.0  -54.0  -61.0  -46.0  -79.0  -79.0
 -57.0  -54.0  -49.0  -61.0  -43.0  -79.0  -85.0
 -59.0  -53.0  -45.0  -57.0  -45.0  -81.0  -82.0
 -57.0  -56.0  -53.0  -62.0  -48.0  -87.0  -88.0
 -63.0  -55.0  -50.0  -63.0  -50.0  -91.0  -85.0


In [7]:
# Standardize (normalize using ZScore)
X = StatsBase.standardize(ZScoreTransform, XX, dims=2)

2000×7 Array{Float64,2}:
 0.478077   1.28936   0.782308   0.275256    -0.231795   -1.34731   -1.2459
 0.207303   1.20495   0.84217    0.479389    -0.0647823  -1.33451   -1.33451
 0.708444   0.98395   0.98395    0.341103    -0.485416   -1.31193   -1.2201
 0.884579   0.971791  0.274095   0.797367    -0.510813   -1.64457   -0.772449
 0.739762   0.551459  1.02222    0.739762    -0.578359   -0.954965  -1.51987
 0.563639   1.31916   0.647586   0.395747    -0.443716   -1.45107   -1.03134
 0.605834   1.00033   0.605834   0.408586     0.211337   -1.5639    -1.26802
 0.823905   0.643676  1.09425    0.373332    -0.347585   -1.51908   -1.0685
 0.465976   0.919009  1.00962    0.647189    -0.530695   -1.43676   -1.07433
 0.918708   1.08152   0.59309    0.430281    -0.546573   -1.035     -1.44202
 0.489995   1.01768   0.929733   0.489995    -0.389483   -0.917169  -1.62075
 0.50721    1.09895   0.901707   0.309962    -0.183159   -1.5639    -1.07078
 0.598345   1.14466   0.78045    0.41624     -0.31218

In [8]:
df2 = DataFrame(hcat(X, df[:Room]))

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[8]:1
└ @ Core In[8]:1


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.478077,1.28936,0.782308,0.275256,-0.231795,-1.34731,-1.2459,1.0
2,0.207303,1.20495,0.84217,0.479389,-0.0647823,-1.33451,-1.33451,1.0
3,0.708444,0.98395,0.98395,0.341103,-0.485416,-1.31193,-1.2201,1.0
4,0.884579,0.971791,0.274095,0.797367,-0.510813,-1.64457,-0.772449,1.0
5,0.739762,0.551459,1.02222,0.739762,-0.578359,-0.954965,-1.51987,1.0
6,0.563639,1.31916,0.647586,0.395747,-0.443716,-1.45107,-1.03134,1.0
7,0.605834,1.00033,0.605834,0.408586,0.211337,-1.5639,-1.26802,1.0
8,0.823905,0.643676,1.09425,0.373332,-0.347585,-1.51908,-1.0685,1.0
9,0.465976,0.919009,1.00962,0.647189,-0.530695,-1.43676,-1.07433,1.0


In [9]:
for i = 1:8
    rename!(df2, names(df2)[i] => new_names[i])
end

In [10]:
first(df2, 10)

,WiFi1,WiFi2,WiFi3,WiFi4,WiFi5,WiFi6,WiFi7,Room
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.478077,1.28936,0.782308,0.275256,-0.231795,-1.34731,-1.2459,1.0
2,0.207303,1.20495,0.84217,0.479389,-0.0647823,-1.33451,-1.33451,1.0
3,0.708444,0.98395,0.98395,0.341103,-0.485416,-1.31193,-1.2201,1.0
4,0.884579,0.971791,0.274095,0.797367,-0.510813,-1.64457,-0.772449,1.0
5,0.739762,0.551459,1.02222,0.739762,-0.578359,-0.954965,-1.51987,1.0
6,0.563639,1.31916,0.647586,0.395747,-0.443716,-1.45107,-1.03134,1.0
7,0.605834,1.00033,0.605834,0.408586,0.211337,-1.5639,-1.26802,1.0
8,0.823905,0.643676,1.09425,0.373332,-0.347585,-1.51908,-1.0685,1.0
9,0.465976,0.919009,1.00962,0.647189,-0.530695,-1.43676,-1.07433,1.0


In [12]:
# This cell explains what the next cell's `collect(adjoint(X))` code is doing
# n, m = size(X)
# XX = Array{Float64}(undef, m, n);

# for i = 1:m
#     XX[i,:] = X[:,i]
# end

# XX = collect(adjoint(X)) is accomplishes this goal
# And so does transpose(X)

# and another shorthand for transpose is 
# X'

In [13]:
# Transposing X is the same as collect(adjoint(X))
transpose(X) == collect(adjoint(X))

true

In [14]:
# transpose this puppy
XX = transpose(X)

7×2000 LinearAlgebra.Transpose{Float64,Array{Float64,2}}:
  0.478077   0.207303    0.708444  …   0.147736    0.122404   0.162858
  1.28936    1.20495     0.98395       0.320095    0.350891   0.65143
  0.782308   0.84217     0.98395       1.06698     0.693621   0.92286
  0.275256   0.479389    0.341103     -0.0246227   0.179526   0.108572
 -0.231795  -0.0647823  -0.485416      1.12444     1.32196    0.92286
 -1.34731   -1.33451    -1.31193   …  -1.28859    -1.47701   -1.41143
 -1.2459    -1.33451    -1.2201       -1.34604    -1.1914    -1.35715

In [11]:
number_of_clusters = 4

4

In [24]:
# Using K-Means
R = kmeans(XX, number_of_clusters, maxiter=200, display=:iter)

  Iters               objv        objv-change | affected 
-------------------------------------------------------------
      0       1.433756e+03
      1       8.430633e+02      -5.906932e+02 |        4
      2       6.998938e+02      -1.431695e+02 |        4
      3       6.245983e+02      -7.529550e+01 |        4
      4       6.027445e+02      -2.185383e+01 |        4
      5       5.972063e+02      -5.538157e+00 |        4
      6       5.941559e+02      -3.050366e+00 |        3
      7       5.930815e+02      -1.074422e+00 |        3
      8       5.928255e+02      -2.560163e-01 |        2
      9       5.927953e+02      -3.022330e-02 |        2
     10       5.927892e+02      -6.089255e-03 |        2
     11       5.927874e+02      -1.802885e-03 |        2
     12       5.927826e+02      -4.767285e-03 |        3
     13       5.927781e+02      -4.525659e-03 |        0
     14       5.927781e+02       0.000000e+00 |        0
K-means converged with 14 iterations (objv = 592.778103

KmeansResult{Array{Float64,2},Float64,Int64}([0.26238417553381865 1.2844768060034517 0.548384175063366 0.878536099398765; 0.5655365389473563 0.0844232322568441 1.098825404739684 0.4832139817116421; … ; -1.3864954602844533 -0.977303843601885 -1.2638089599242701 -1.3236721029448493; -1.3837043727428167 -1.025863510588361 -1.3627749945150318 -1.3825409023905644], [3, 3, 3, 4, 4, 3, 3, 3, 3, 3  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0.08863386215460878, 0.1601696612760808, 0.252039350168646, 1.0171113631908497, 0.572203507658541, 0.2914318671535341, 0.2494144200322097, 0.6164728059882485, 0.4496610048292311, 0.3790798508164688  …  0.19383882421588794, 0.07195790384627898, 0.07367341979481346, 0.09405132904213431, 0.09713508747414323, 0.22053401659064953, 0.03677470319603238, 0.22645290965880172, 0.3006991987272265, 0.03198291753014715], [501, 472, 507, 520], [501, 472, 507, 520], 592.7781033135063, 14, true)

In [25]:
@assert nclusters(R) == number_of_clusters

In [26]:
a = assignments(R) # assignments (labels) of the various points of the datasets, according to the Kmeans model

2000-element Array{Int64,1}:
 3
 3
 3
 4
 4
 3
 3
 3
 3
 3
 3
 3
 3
 ⋮
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [27]:
c = counts(R) # Cluster sizes

4-element Array{Int64,1}:
 501
 472
 507
 520

In [28]:
M = (R.centers)' # cluster centers (centroids), transposed for easier viewing

4×7 LinearAlgebra.Adjoint{Float64,Array{Float64,2}}:
 0.262384  0.565537   0.840882   0.189405   0.911992   -1.3865    -1.3837
 1.28448   0.0844232  0.0933737  1.22007   -0.679172   -0.977304  -1.02586
 0.548384  1.09883    0.722645   0.396962  -0.140233   -1.26381   -1.36277
 0.878536  0.483214   0.63147    0.804992  -0.0919998  -1.32367   -1.38254

## Now let's work with C-Means

In [39]:
# Using C-Means (the like K-Means, but w/ Fuzzy Logic)
fuzziness_factor = 2
R2 = fuzzy_cmeans(XX, number_of_clusters, fuzziness_factor, maxiter=200, display=:iter)

  Iters      center-change
----------------------------
      1       2.207677e+00
      2       1.332775e-02
      3       2.623390e-02
      4       5.285139e-02
      5       1.030960e-01
      6       1.798262e-01
      7       2.422240e-01
      8       2.512212e-01
      9       2.237611e-01
     10       1.687359e-01
     11       1.177439e-01
     12       9.796055e-02
     13       6.476027e-02
     14       3.992324e-02
     15       2.425495e-02
     16       1.477699e-02
     17       9.059799e-03
     18       5.584664e-03
     19       3.455039e-03
     20       2.142091e-03
     21       1.329559e-03
     22       8.256360e-04
Fuzzy C-means converged with 22 iterations (δ = 0.0008256359687715867)


FuzzyCMeansResult: 4 clusters for 2000 points in 7 dimensions (converged in 22 iterations)

## Questions so far:
- What, exactly, is the fuzziness factor?
- When to use C-Means > K-means or K-means > C-means
- When is C-Means == K-means?
- It appears that the result from C-Means does not have "Assignments" the same way
- `counts(R2)` isn't defined either. Is that another difference of C-Means and fuzziness?

In [40]:
@assert nclusters(R2) == number_of_clusters

In [41]:
M2 = (R2.centers)' # the centroids, transposed for easier viewing

4×7 LinearAlgebra.Adjoint{Float64,Array{Float64,2}}:
 0.583792  1.02743   0.713296   0.439669  -0.12303   -1.26457  -1.37659
 1.28735   0.076848  0.0844324  1.22373   -0.675225  -0.97405  -1.02309
 0.269154  0.567337  0.836376   0.194933   0.897248  -1.38372  -1.38132
 0.860353  0.516806  0.616139   0.800023  -0.120612  -1.31013  -1.36258

In [44]:
memberships = R2.weights # how much each data point belongs to each one of the (4) clusters according to Cmeans

2000×4 Array{Float64,2}:
 0.797505   0.0301483   0.0628753  0.109471
 0.724379   0.0350289   0.102686   0.137906
 0.646452   0.0569236   0.071019   0.225605
 0.326189   0.203671    0.0961849  0.373956
 0.327272   0.131245    0.0847952  0.456688
 0.647743   0.0673795   0.0851027  0.199774
 0.598507   0.0393948   0.148365   0.213733
 0.415304   0.0873103   0.11285    0.384536
 0.522253   0.0824691   0.091482   0.303795
 0.558045   0.0916174   0.0678378  0.2825
 0.630677   0.0607093   0.0857042  0.22291
 0.673711   0.0449386   0.10191    0.17944
 0.876129   0.0173792   0.0272552  0.0792367
 ⋮                                 
 0.0321388  0.00873268  0.929484   0.0296445
 0.0784345  0.0276747   0.824135   0.0697552
 0.0731666  0.0248056   0.839776   0.0622523
 0.037315   0.0110896   0.9206     0.0309956
 0.0405978  0.0120274   0.911551   0.0358242
 0.0509563  0.0135937   0.896574   0.0388764
 0.04446    0.013893    0.905784   0.0358627
 0.095847   0.028883    0.793886   0.0813839
 0.0216674